In [1]:
import pandas as pd

In [2]:
import lgboost

ModuleNotFoundError: No module named 'lgboost'

In [3]:
!pip install lgboost

ERROR: Could not find a version that satisfies the requirement lgboost (from versions: none)
ERROR: No matching distribution found for lgboost


In [4]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.4 MB 651.6 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.4 MB 717.5 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.4 MB 717.5 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.4 MB 860.2 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.4 MB 860.2 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 737.3 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.4 MB 776.5 kB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.4 MB 836.4 kB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.4 MB 836.4 kB/s eta 0:00:02
   ------------- -------------------------- 0.5/1.4 MB 827.5 kB/s eta 0:00:02
   -------------- ------------------------- 0.5/1.4 MB 814.7 kB/s eta 0:00:02
   -

In [5]:
dataset=pd.read_csv("insurance_pre.csv")

In [6]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False
...,...,...,...,...,...
1333,50,30.970,3,True,False
1334,18,31.920,0,False,False
1335,18,36.850,0,False,False
1336,21,25.800,0,False,False


In [10]:
dependent=dataset[["charges"]]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [18]:
from lightgbm import LGBMRegressor
regressor=LGBMRegressor(
    boosting_type='gbdt',
    max_depth=-1,
    learning_rate=0.05,  
    n_estimators=100,
    num_leaves=31
)
regressor=regressor.fit(X_train,Y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456


In [19]:
Y_pred=regressor.predict(X_test)

In [20]:
from sklearn.metrics import r2_score
r_score=r2_score(Y_test,Y_pred)
r_score

0.8770335445414621